In [9]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
URL_YANDES_NEWS = 'https://yandex.ru/news/'
URL_LENTA_NEWS = 'https://lenta.ru/'
URL_DZEN_NEWS = 'https://dzen.ru/news/'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

params = {
    'sso_failed': '',
}

In [4]:
def get_content_dom_from_html_text(url, headers=None, params=None):
    response = requests.get(url, headers=headers, params=params)
    content_dom = html.fromstring(response.text)

    print(response.url)
    
    return content_dom


def parser_yandex_news(content_dom):
    news_container = content_dom.xpath("//section//div[contains(@class, 'mg-grid__item')]")
    yandex_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//div[@class='mg-card-footer__left']//a/text()")[0]
        new_text = new_container.xpath(".//h2/a/text()")[0].replace('\xa0', ' ')
        new_link = new_container.xpath(".//h2/a/@href")[0]
        new_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        yandex_news.append(news_dict)

    return yandex_news


def parser_dzen_news(content_dom):
    news_container = content_dom.xpath("//div[contains(@class,'mg-card ')]")
    dzen_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//a[@class='mg-card__source-link']/text()")[0]
        new_text = new_container.xpath(".//h2/a/text()")[0].replace('\xa0', ' ')
        new_link = new_container.xpath(".//h2/a/@href")[0]
        new_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        dzen_news.append(news_dict) 

    return dzen_news


def parser_lenta_news(content_dom):
    news_container = content_dom.xpath("//a[contains(@class, 'card-')]")
    lenta_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//*[name()='svg']/*[name()='use']/attribute::*")
        if len(new_source) == 1:
            new_source = new_source[0].split('ui-label_')
            if len(new_source) == 2:
                new_source = new_source[1]
            else:
                new_source = 'Lenta.ru'
        else:
            new_source = 'Lenta.ru' 
        new_text = new_container.xpath(".//span[contains(@class, 'card-')]/text()")
        if len(new_text) == 1:
            new_text = new_text[0]
        else:
            new_text = None  
        new_link = URL_LENTA_NEWS + new_container.xpath("./@href")[0]
        
        new_date = new_container.xpath(".//time/text()")
        if len(new_date) == 1:
            new_date = new_date[0]
        else:
            new_date = None
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        lenta_news.append(news_dict)
        
    return lenta_news


def get_news(content_dom, parser_name='dzen'):
    result = None
    
    if parser_name == 'dzen':
      result = parser_dzen_news(content_dom)
    elif parser_name == 'yandex':
      result = parser_yandex_news(content_dom)
    elif parser_name == 'lenta':
      result = parser_lenta_news(content_dom)
    
    return result

In [5]:
dom = get_content_dom_from_html_text(URL_DZEN_NEWS, headers=headers, params=params)

https://dzen.ru/news?issue_tld=ru&sso_failed=


In [6]:
dzen_news = get_news(dom, 'dzen')
dzen_news

[{'new_source': 'Lenta.ru',
  'new_text': 'Путин потребовал исправить ошибки при частичной мобилизации и больше их не допускать',
  'new_link': 'https://dzen.ru/news/story/Putin_potreboval_ispravit_oshibki_prichastichnoj_mobilizacii_i_bolshe_ikh_ne_dopuskat--550772fff0d5c22d8c46b328d02dc482?lang=ru&rubric=index&fan=1&stid=jC6aymENnsaG8kw67YDb&t=1664478573&tt=true&persistent_id=229918354&story=29b840b9-7094-56d5-8938-1ca2b936d5e4&issue_tld=ru',
  'new_date': '14:56'},
 {'new_source': 'РИА Новости',
  'new_text': 'South Stream Transport: из-за санкций ЕС у оператора «Турецкого потока» отозвали лицензию',
  'new_link': 'https://dzen.ru/news/story/South_Stream_Transport_iz-zasankcij_ES_uoperatora_Tureckogo_potoka_otozvali_licenziyu--0a56cf430ebae05f4305531ac8c6e033?lang=ru&rubric=index&fan=1&stid=wR9a7DNBxuDQ5E6osE1C&t=1664478573&tt=true&persistent_id=229908583&story=40a67d43-19f2-5c48-997a-3ab8dc4d77ae&issue_tld=ru',
  'new_date': '15:09'},
 {'new_source': 'РИА Новости',
  'new_text': 'Фи

In [10]:
pd.DataFrame(dzen_news)

,new_source,new_text,new_link,new_date
0,Lenta.ru,Путин потребовал исправить ошибки при частично...,https://dzen.ru/news/story/Putin_potreboval_is...,14:56
1,РИА Новости,South Stream Transport: из-за санкций ЕС у опе...,https://dzen.ru/news/story/South_Stream_Transp...,15:09
2,РИА Новости,Финляндия закроет границу для туристов из Росс...,https://dzen.ru/news/story/Finlyandiya_zakroet...,14:40
3,Lenta.ru,Путин обсудил по телефону с президентом Турции...,https://dzen.ru/news/story/Putin_obsudil_potel...,15:03
4,RT на русском,Подписание договоров о вхождении в состав Росс...,https://dzen.ru/news/story/Podpisanie_dogovoro...,14:21
...,...,...,...,...
60,RT на русском,Стоимость нового внедорожника Aurus Komendant ...,https://dzen.ru/news/story/Stoimost_novogo_vne...,15:01
61,Авторевю,В Липецкой области началось производство элект...,https://dzen.ru/news/story/VLipeckoj_oblasti_n...,10:00
62,Автоновости дня,АВТОВАЗ планирует возобновить производство авт...,https://dzen.ru/news/story/AVTOVAZ_planiruet_v...,12:12
63,Автоновости дня,Компания HAVAL планирует представить 7 новых м...,https://dzen.ru/news/story/Kompaniya_HAVAL_pla...,14:49


In [7]:
dom, response = get_content_dom_from_html_text(URL_LENTA_NEWS, headers=headers)

https://lenta.ru/


In [8]:
get_news(dom, 'lenta')

[{'new_source': 'Lenta.ru',
  'new_text': None,
  'new_link': 'https://lenta.ru//news/2022/09/29/suetaaa/',
  'new_date': '20:24'},
 {'new_source': 'Lenta.ru',
  'new_text': 'В России представили новый внедорожник класса люкс Aurus Komendant',
  'new_link': 'https://lenta.ru//news/2022/09/29/komandant/',
  'new_date': '22:08'},
 {'new_source': 'Lenta.ru',
  'new_text': 'На Украине прокомментировали запрет на применение оружия США',
  'new_link': 'https://lenta.ru//news/2022/09/29/gun/',
  'new_date': '21:57'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Блинкен заявил об отказе США признать референдумы и продолжении помощи Киеву',
  'new_link': 'https://lenta.ru//news/2022/09/29/blinken/',
  'new_date': '21:53'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Составлен топ-5 городов Подмосковья с доступным жильем в новостройках',
  'new_link': 'https://lenta.ru/https://moslenta.ru/news/city/dostupnoe-zhile-v-novostroikakh-29-09-2022.htm/',
  'new_date': '21:49'},
 {'new_source': 'Lenta.ru',